In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import math

In [2]:
data_dir="D:\\programs\\ads_flask\\logres_op\\data\\"
model_dir="D:\\programs\\ads_flask\\logres_op\\model\\"

In [3]:
ld=[]
files=[]
for root, dirs, files in os.walk(data_dir, topdown=False):
   for name in files:
      ld.append(os.path.join(root, name))
for i in range(len(files)):
   files[i]=files[i].replace(".csv","")

In [4]:
lm=[]
files=[]
for root, dirs, files in os.walk(model_dir, topdown=False):
   for name in files:
      lm.append(os.path.join(root, name))
for i in range(len(files)):
   files[i]=files[i].replace(".h5","")

In [5]:
data_list=[]
def conv_logret_close(y,val):
    yc=[]
    for i in range(len(y)):
        if i==0:
            yc.append((math.e**y)*val)
        else:
            yc.append((math.e**y)*yc[i-1]) 
    return np.array(yc).squeeze()

In [6]:
for i in range(len(files)):
    df=pd.read_csv(ld[i])
    m=tf.keras.models.load_model(lm[i])
    x=np.array(df["logret"][df["logret"].size-91:df["logret"].size-1])
    x=x.reshape(1,x.shape[0],1)
    y=m.predict(x)
    y=conv_logret_close(y,df["close "][df["OPEN "].size-1])
    data_list.append([files[i],df["logret"].sum()/df["logret"].size,y[89]-float(df["close "].tail(1)),(y[89]-float(df["close "].tail(1)))/float(df["close "].tail(1))])

1/1 [==============================] - 3s 3s/step


In [8]:
d=pd.DataFrame(data_list,columns=["stock",'10Y_mean_logret',"expected gain","pct change"])
d.nlargest(10,'10Y_mean_logret'),d.nlargest(10,"pct change")

(         stock  10Y_mean_logret  expected gain  pct change
 0     ADANIENT         0.001746      32.103320    0.013294
 46       TITAN         0.001042      -2.479004   -0.000834
 10   BRITANNIA         0.000823       1.612988    0.000319
 41  TATACONSUM         0.000740       0.462732    0.000539
 31      MARUTI         0.000727      10.459570    0.001102
 2   APOLLOHOSP         0.000671       0.941211    0.000182
 7   BAJFINANCE         0.000657      20.669727    0.002851
 48         UPL         0.000649       1.944409    0.002845
 1   ADANIPORTS         0.000637      -0.019055   -0.000026
 33   NESTLEIND         0.000611      48.108203    0.002096,
          stock  10Y_mean_logret  expected gain  pct change
 0     ADANIENT         0.001746      32.103320    0.013294
 7   BAJFINANCE         0.000657      20.669727    0.002851
 48         UPL         0.000649       1.944409    0.002845
 14     DRREDDY         0.000337      12.772168    0.002606
 25  INDUSINDBK         0.000413       

In [10]:
x=d[["stock","10Y_mean_logret"]]

In [12]:
x.to_csv("10Ymeanlogrte.csv")